In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import face_recognition
import os
from datetime import datetime

In [13]:
#now create a list of all the available images
path = 'ImageAttendance'
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['hanan.jpeg', 'Naveed.jpeg']


In [14]:
#now we will loop through all the images and encode them
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['hanan', 'Naveed']


In [15]:
#now we will encode all the images
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [16]:
#check the above function
encodeListKnown = findEncodings(images)
print(len(encodeListKnown))

2


In [17]:
#now we will mark the attendance
def markAttendance(name):
    with open('Attendance.csv','r+') as f: #r+ means read and write
        myDataList = f.readlines() #read all the lines
        nameList = []   #create a list of names
        for line in myDataList: #loop through all the lines
            entry = line.split(',') #split the line at comma
            nameList.append(entry[0])   #append the first element of the list
        if name not in nameList:   #if name is not in the list
            now = datetime.now()   #get the current time
            dtString = now.strftime('%H:%M:%S')  #convert the time to string
            f.writelines(f'n{name},{dtString}') #write the name and time in the file with a new line 


In [18]:
#initialize the webcam

cap = cv2.VideoCapture(0)

#now we will loop through the frames
while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    #now we will find the faces in the current frame
    
    facesCurFrame = face_recognition.face_locations(imgS) #this will return the list of all the faces in the current frame
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame) #this will return the list of all the encodings of the faces in the current frame

    #find the matches
    #now we will loop through all the encodings of the faces in the current frame
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame): #it will grab the encoding and the location of the face
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace) #this will return the list of all the matches
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace) #this will return the list of all the distances
        #print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4 #since we have resized the image to 1/4th of its original size
            cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2) #this will draw a rectangle around the face
            cv2.rectangle(img, (x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED) #this will draw a rectangle below the face
            cv2.putText(img, name, (x1+6,y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2) #this will write the name of the person
            markAttendance(name) #this will mark the attendance of the person

    cv2.imshow('Webcam', img)
    cv2.waitKey(1)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
